# Get Financial Research Report

In [ ]:
import pandas as pd
import atrader as at
import numpy as np
from eastmoney_scraper_many1 import EastmoneyReportScraper

In [ ]:
def get_eastmoney_data(start_date="2025-04-09 00:00:00", end_date="2025-04-10 00:00:00", output_file="eastmoney_report_data.xlsx", desired_count=1):
    # Initialize the crawler (custom cookies can be passed in)
    scraper = EastmoneyReportScraper()
    scraper.scrape( start_date, end_date, output_file, desired_count=desired_count)

In [ ]:
if __name__ == '__main__':
    keyword =[['期货'],['股票'],['股指']] #keysords futures, stock, index
    start_date = '2025-05-24'
    end_date = '2025-05-31'
    output_file = 'eastmoney_report_data.xlsx'
    desired_count = 100

    get_eastmoney_data(start_date, end_date, output_file, desired_count=desired_count)

# Generate sentiment dictionary

In [ ]:
import os
import re
import json
from datetime import datetime
from collections import defaultdict
import jieba
import numpy as np
from snownlp import SnowNLP
from gensim.models import KeyedVectors
import PyPDF2

Read the research report from the folder

In [ ]:
def read_reports_from_folder(folder_path):
    reports = []
    # Use the os.walk function to traverse the specified folder and its subfolders
    # The os.walk function returns a tuple (root, dirs, files)
    # root represents the path of the current directory being traversed
    # dirs is a list of subdirectory names in the current directory
    # files is a list containing the names of the files in the current directory
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.pdf'):
                file_path = os.path.join(root, file)  # Combine multiple path components into a valid path string
                try:
                    with open(file_path, 'rb') as f:  # Note the use of binary mode 'rb'
                        reader = PyPDF2.PdfReader(f)
                        content = ""
                        for page in reader.pages:
                            content += page.extract_text()
                        reports.append({
                            'filename': file,
                            'content': content
                        })
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")
    return reports

Processing of clauses

In [ ]:
def preprocess_text(text):
    """
    Preprocess text and split it into sentences
    1. Remove special characters (retain Chinese, English, and numbers)
    2. Remove extra spaces
    3. Chinese sentence segmentation (based on common punctuation)
    """
    # Remove special characters (retain Chinese, English, numbers, and basic punctuation)
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9,.!?;:\s]', ' ', text)
    
    # Remove extra spaces, \s+ matches one or more consecutive whitespace characters
    text = re.sub(r'\s+', ' ', text).strip()    
    
    # Chinese sentence segmentation (split by common punctuation)
    sentences = re.split(r'([.!?;:])', text)
    
    # Recombine sentences (attach punctuation back to the previous sentence)
    processed_sentences = []
    for i in range(0, len(sentences)-1, 2):
        sentence = sentences[i] + (sentences[i+1] if i+1 < len(sentences) else '')
        if sentence.strip():  # Remove empty sentences
            processed_sentences.append(sentence.strip())
    
    # Handle the last element if there's an odd number of elements
    if len(sentences) % 2 != 0 and sentences[-1].strip():
        processed_sentences.append(sentences[-1].strip())
    
    return processed_sentences

Initially construct financial sentiment dictionary

In [ ]:
def is_chinese_word(word):
    """Check if a word consists purely of Chinese characters"""
    for char in word:
        if not '\u4e00' <= char <= '\u9fff':
            return False
    return True

def build_sentiment_dictionary(positive_texts, negative_texts, min_freq=3):
    """
    Build a sentiment dictionary (only retaining pure Chinese words)
    positive_texts: List of positive texts
    negative_texts: List of negative texts
    min_freq: Minimum frequency threshold
    """
    # Create frequency dictionaries
    pos_word_freq = defaultdict(int)
    neg_word_freq = defaultdict(int)
    
    # Analyze positive texts
    for text in positive_texts:
        words = jieba.lcut(text)
        for word in words:
            # Only retain pure Chinese words longer than 1 character
            if len(word) > 1 and is_chinese_word(word):
                pos_word_freq[word] += 1
    
    # Analyze negative texts
    for text in negative_texts:
        words = jieba.lcut(text)
        for word in words:
            # Only retain pure Chinese words longer than 1 character
            if len(word) > 1 and is_chinese_word(word):
                neg_word_freq[word] += 1
    
    # Build sentiment dictionary
    sentiment_dict = {}
    
    # Add positive words
    for word, freq in pos_word_freq.items():
        if freq >= min_freq and word not in sentiment_dict:
            sentiment_dict[word] = {
                'sentiment': 'positive',
                'frequency': freq,
                'source': 'auto_generated'
            }
    
    # Add negative words
    for word, freq in neg_word_freq.items():
        if freq >= min_freq:
            if word in sentiment_dict:
                # If a word appears in both positive and negative texts, compare frequencies
                if freq > sentiment_dict[word]['frequency']:
                    sentiment_dict[word] = {
                        'sentiment': 'negative',
                        'frequency': freq,
                        'source': 'auto_generated'
                    }
            else:
                sentiment_dict[word] = {
                    'sentiment': 'negative',
                    'frequency': freq,
                    'source': 'auto_generated'
                }
    
    return sentiment_dict

Preliminary labeling of emotion labels

In [ ]:
if __name__ == '__main__':
    output_file = '2025-05-24_2025-05-31'
    
    # Create save directory
    save_dir = os.path.join(output_file, 'sentiment_results')
    os.makedirs(save_dir, exist_ok=True)
    
    # Initialize storage
    positive_texts = []
    negative_texts = []
    
    # Read research reports from folder
    reports = read_reports_from_folder(output_file)
    processed_reports = []
    
    for report in reports:
        # Preprocess each report's content into sentences
        processed_content = preprocess_text(report['content'])
        processed_reports.append({
            'filename': report['filename'],
            'content': processed_content  # This is a list of sentences
        })

    # Perform sentiment analysis and classify texts
    for report in processed_reports:
        print(f"\nAnalyzing file: {report['filename']}")
        
        # Create separate positive and negative result files for each report
        report_name = os.path.splitext(report['filename'])[0]
        pos_file = os.path.join(save_dir, f"{report_name}_positive.txt")
        neg_file = os.path.join(save_dir, f"{report_name}_negative.txt")
        
        # Calculate sentiment scores using SnowNLP for each sentence in the report
        # Classify sentences as positive or negative and save to corresponding files
        with open(pos_file, 'w', encoding='utf-8') as f_pos, \
             open(neg_file, 'w', encoding='utf-8') as f_neg:
            
            for i, sentence in enumerate(report['content'], 1):
                if len(sentence) < 5:
                    continue
                    
                try:
                    s = SnowNLP(sentence)
                    sentiment_score = s.sentiments
                    
                    if sentiment_score >= 0.7:
                        f_pos.write(f"{sentiment_score:.4f}\t{sentence}\n")
                        positive_texts.append(sentence)
                    elif sentiment_score <= 0.3:
                        f_neg.write(f"{sentiment_score:.4f}\t{sentence}\n")
                        negative_texts.append(sentence)
                        
                except Exception as e:
                    print(f"Error analyzing sentence {i}: {str(e)}")
                    continue
    
    # Build sentiment dictionary based on word frequency in positive and negative texts
    sentiment_dict = build_sentiment_dictionary(positive_texts, negative_texts)
    
    # Save sentiment dictionary
    dict_file = os.path.join(save_dir, 'financial_sentiment_dict.json')
    with open(dict_file, 'w', encoding='utf-8') as f:
        json.dump(sentiment_dict, f, ensure_ascii=False, indent=2)    # json.dump() converts a Python object (usually a dictionary or list) into a JSON-formatted string. Setting ensure_ascii=False ensures non-ASCII characters (like Chinese) are output correctly.
    
    print("\nSentiment dictionary construction completed:")
    print(f"- Number of positive texts: {len(positive_texts)}")
    print(f"- Number of negative texts: {len(negative_texts)}")
    print(f"- Size of sentiment dictionary: {len(sentiment_dict)}")
    print(f"Results saved to: {save_dir}")

# Augmented sentiment dictionary

In [ ]:
import bz2

# Path to the compressed file
compressed_file = 'sgns.financial.word.bz2'
# Path to the decompressed file
decompressed_file = 'sgns.financial.word'

with bz2.open(compressed_file, 'rb') as f_in:       # 'rb' indicates opening in read-only binary mode
    with open(decompressed_file, 'wb') as f_out:   # 'wb' indicates opening in write binary mode
        f_out.write(f_in.read())        # Read all content from the compressed file and write it to the decompressed file

print("File decompression completed.")

# Dictionary construction and extension

In [ ]:
def expand_sentiment_dictionary(sentiment_dict, reports, word2vec_model, min_similarity=0.5):
    """
    Expand the sentiment dictionary by analyzing sentiment words not covered in the initial dictionary
    using the WORD2VEC word vector model to identify similar terms in research reports.
    
    sentiment_dict: Initial sentiment dictionary
    reports: List of processed research reports
    word2vec_model: Word2Vec word vector model
    min_similarity: Minimum similarity threshold
    return: Expanded sentiment dictionary
    """
    new_sentiment_dict = sentiment_dict.copy()
    for report in reports:
        for sentence in report['content']:
            words = jieba.lcut(sentence)
            for word in words:
                if len(word) > 1 and is_chinese_word(word) and word not in new_sentiment_dict:
                    similar_words = []
                    try:
                        # Find words similar to known sentiment terms
                        for known_word in sentiment_dict.keys():
                            if word2vec_model.has_index_for(known_word) and word2vec_model.has_index_for(word):  # has_index_for checks word vector existence
                                similarity = word2vec_model.similarity(known_word, word)
                                if similarity >= min_similarity:
                                    similar_words.append((known_word, similarity))
                    except KeyError:
                        continue

                    if similar_words:
                        # Determine sentiment of new word based on similar terms
                        pos_count = sum([1 for known_word, _ in similar_words if sentiment_dict[known_word]['sentiment'] == 'positive'])
                        neg_count = sum([1 for known_word, _ in similar_words if sentiment_dict[known_word]['sentiment'] == 'negative'])
                        if pos_count > neg_count:
                            new_sentiment_dict[word] = {
                               'sentiment': 'positive',
                                'frequency': 1,
                               'source': 'expanded_by_word2vec'
                            }
                        elif neg_count > pos_count:
                            new_sentiment_dict[word] = {
                               'sentiment': 'negative',
                                'frequency': 1,
                               'source': 'expanded_by_word2vec'
                            }
                        else:
                            new_sentiment_dict[word] = {
                               'sentiment': 'neutral',
                                'frequency': 1,
                               'source': 'expanded_by_word2vec'
                            }
                    else:
                        new_sentiment_dict[word] = {
                           'sentiment': 'neutral',
                            'frequency': 1,
                           'source': 'new_word'
                        }
    return new_sentiment_dict

In [ ]:
def adjust_sentiment_weights(sentiment_dict, reports):
    """
    Adjust sentiment weights of words in the dictionary based on their contextual sentiment performance
    
    sentiment_dict: Sentiment dictionary
    reports: List of processed research reports
    return: Sentiment dictionary with adjusted weights
    """
    for report in reports:
        for sentence in report['content']:
            words = jieba.lcut(sentence)
            s = SnowNLP(sentence)
            sentiment_score = s.sentiments
            for word in words:
                if word in sentiment_dict:
                    if sentiment_score >= 0.7:
                        if sentiment_dict[word]['sentiment'] == 'positive':
                            sentiment_dict[word]['frequency'] += 1
                    elif sentiment_score <= 0.3:
                        if sentiment_dict[word]['sentiment'] == 'negative':
                            sentiment_dict[word]['frequency'] += 1
    return sentiment_dict

In [ ]:
if __name__ == '__main__':

    # Load Word2Vec word vector model
    word2vec_model = KeyedVectors.load_word2vec_format('sgns.financial.word', binary=False)

    # Expand sentiment dictionary (if uncovered words have >80% similarity with existing sentiment words in Word2Vec vectors,
    # they are counted as similar words. Determine sentiment based on similar words - if more positive than negative,
    # classify as positive and add to the dictionary)
    sentiment_dict = expand_sentiment_dictionary(sentiment_dict, processed_reports, word2vec_model)

    # Adjust sentiment weights (use SnowNLP to determine sentiment scores: words in sentences with score ≥0.7 are considered
    # positive, those in sentences with score ≤0.3 are considered negative; increase their frequency accordingly)
    sentiment_dict = adjust_sentiment_weights(sentiment_dict, processed_reports)

    # Save sentiment dictionary
    dict_file = os.path.join(save_dir, 'financial_sentiment_dict_0.5.json')
    with open(dict_file, 'w', encoding='utf-8') as f:
        json.dump(sentiment_dict, f, ensure_ascii=False, indent=2)   # json.dump() converts Python objects (usually dictionaries or lists) to JSON-formatted strings

    print("\nSentiment dictionary construction completed:")
    print(f"- Number of positive texts: {len(positive_texts)}")
    print(f"- Number of negative texts: {len(negative_texts)}")
    print(f"- Size of sentiment dictionary: {len(sentiment_dict)}")
    print(f"Results saved to: {save_dir}")

# Sentiment Analysis and Prediction Pipeline for Financial Market Data

In [ ]:
# ---------------------- 0. Configurable Keyword ----------------------
KEYWORD = "Corn"  # e.g："China_Unicom"、"AMZN"、"Corn"、"CSI100" 


# ---------------------- 1. Import Dependent Libraries ----------------------
import pandas as pd
import jieba
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from gensim.models import Word2Vec
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm
import json
from pathlib import Path
import torch.nn as nn 
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


# ---------------------- 2. Hyperparameter Configuration ----------------------
# Path configuration (modify according to actual paths)
DATA_PATH = f"data/{KEYWORD}_stock_bar_data_2020-2024.xlsx"  # AMZN_stock_bar_data_2017-2023.xlsx
DICT_PATH = "financial_sentiment_dict_100.json"           # Sentiment dictionary path
STOPWORDS_PATH = "data/stopwords.txt"   # Stopwords list path
OUTPUT_PATH = f"data/{KEYWORD}_data_with_sentiment_scores.csv"    # Final output path
MODEL_SAVE_PATH = f"data/{KEYWORD}_best_sentiment_model.pth"   # Model save path

# Model parameters
EMBED_DIM = 100  # Word embedding dimension
HIDDEN_DIM = 256  # LSTM hidden layer dimension
BATCH_SIZE = 32  # Batch size
EPOCHS = 30  # Training epochs
MAX_LEN = 20  # Maximum text length (titles are usually short)
EARLY_STOPPING_PATIENCE = 3  # Early stopping patience



# ---------------------- 3. Tool Function Definitions ----------------------
def load_stopwords(path):
    """Load stopwords list"""
    
    with open(path, 'r', encoding='utf-8') as f:
        return set(f.read().splitlines())

def preprocess_text(text, stopwords):
    """Text preprocessing: word segmentation + stopword filtering"""
    text = str(text).strip()
    if not text:
        return []
    words = jieba.lcut(text)
    # Retain single-character sentiment words (e.g., "rise", "fall"), only filter meaningless stopwords
    return [word for word in words if word not in stopwords or len(word) >= 1]

def load_sentiment_dict(path):
    """Load sentiment dictionary and convert to {word: (sentiment tendency, frequency)}"""
    with open(path, 'r', encoding='utf-8') as f:
        raw_dict = json.load(f)
    return {word: (info['sentiment'], info['frequency']) for word, info in raw_dict.items()}

def generate_pseudo_labels(tokens, sent_dict):
    """Generate pseudo-labels based on the number of sentiment words (positive/negative/neutral)"""
    pos_count = sum(1 for word in tokens if sent_dict.get(word, (None,))[0] == 'positive')
    neg_count = sum(1 for word in tokens if sent_dict.get(word, (None,))[0] == 'negative')
    if pos_count > neg_count:
        return 2  # Positive
    elif neg_count > pos_count:
        return 1  # Negative
    else:
        return 0  # Neutral


# ---------------------- 4. Data Loading and Preprocessing ----------------------
# 1. Load base data
print("===== Data Loading and Cleaning =====")
try:
    if Path(DATA_PATH).suffix == '.xlsx':
        df = pd.read_excel(DATA_PATH)
    else:
        df = pd.read_csv(DATA_PATH)
except FileNotFoundError:
    raise ValueError(f"File not found: {DATA_PATH}, please check the path")

# 2. Data cleaning
df = df.dropna(subset=['标题']).drop_duplicates(subset=['标题']).reset_index(drop=True)
print(f"Valid data volume after cleaning: {len(df)}")

# 3. Load stopwords and sentiment dictionary
stopwords = load_stopwords(STOPWORDS_PATH)
sent_dict = load_sentiment_dict(DICT_PATH)

# 4. Text preprocessing
df['tokens'] = df['标题'].apply(lambda x: preprocess_text(x, stopwords))
df = df[df['tokens'].apply(len) > 0].reset_index(drop=True)  # Filter empty tokenization results
print(f"Valid tokenized data volume: {len(df)}")

# 5. Generate pseudo-labels (replace random labels)
df['label'] = df['tokens'].apply(lambda x: generate_pseudo_labels(x, sent_dict))
print("Label distribution:")
print(df['label'].value_counts())


# ---------------------- 5. Word Embedding Training ----------------------
print("\n===== Training Word2Vec Word Embeddings =====")
sentences = df['tokens'].tolist()
w2v_model = Word2Vec(
    sentences,
    vector_size=EMBED_DIM,
    window=5,
    min_count=1,
    workers=4,
    sg=1  # Use Skip-gram model
)
vocab = w2v_model.wv.key_to_index
word2idx = {word: idx+1 for idx, word in enumerate(vocab)}  # Index starts from 1 (0 is PAD)
word2idx['<PAD>'] = 0
print(f"Vocabulary size: {len(word2idx)}")


# ---------------------- 6. Build Dataset (Incorporating Sentiment Features) ----------------------
class SentimentDataset(Dataset):
    def __init__(self, df, word2idx, sent_dict, max_len):
        # 1. Word index sequences
        self.input_ids = [
            [word2idx.get(word, 0) for word in tokens] + [0]*(max_len - len(tokens)) 
            if len(tokens) < max_len else [word2idx.get(word, 0) for word in tokens[:max_len]]
            for tokens in df['tokens']
        ]
        self.input_ids = torch.LongTensor(self.input_ids)
        
        # 2. Sentiment statistical features (number of positive words, total frequency of negative words)
        self.pos_counts = torch.FloatTensor([
            sum(1 for word in tokens if sent_dict.get(word, (None,))[0] == 'positive')
            for tokens in df['tokens']
        ]).unsqueeze(1)  # Shape: (n, 1)
        
        self.neg_weights = torch.FloatTensor([
            sum(sent_dict[word][1] for word in tokens if sent_dict.get(word, (None,))[0] == 'negative')
            for tokens in df['tokens']
        ]).unsqueeze(1)  # Shape: (n, 1)
        
        # 3. Labels
        self.labels = torch.LongTensor(df['label'].values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'pos_count': self.pos_counts[idx],
            'neg_weight': self.neg_weights[idx],
            'label': self.labels[idx]
        }


# ---------------------- 7. Define LSTM Model with Sentiment Features ----------------------
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, sent_feat_dim=2):
        super().__init__()
        # Word embedding layer (using pre-trained Word2Vec)
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        # LSTM layer (bidirectional)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.2
        )
        
        # Sentiment feature processing layer (map 2D sentiment features to LSTM output dimension)
        self.sent_fc = nn.Linear(sent_feat_dim, hidden_dim * 2)  # Bidirectional LSTM output dimension is hidden_dim*2
        
        # Classification head (fuse LSTM features and sentiment features)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2 + hidden_dim * 2, 128),  # Concatenate LSTM output and sentiment features
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        # Word embedding: (batch_size, seq_len, embed_dim)
        embeds = self.embedding(x['input_ids'])
        
        # LSTM processing: out shape (batch_size, seq_len, hidden_dim*2)
        out, _ = self.lstm(embeds)
        lstm_feat = out[:, -1, :]  # Take hidden state of the last time step
        
        # Sentiment feature processing: (batch_size, 2) → (batch_size, hidden_dim*2)
        sent_feat = torch.cat([x['pos_count'], x['neg_weight']], dim=1)
        sent_feat = self.sent_fc(sent_feat)
        
        # Fuse features and classify
        combined_feat = torch.cat([lstm_feat, sent_feat], dim=1)
        return self.classifier(combined_feat)


# ---------------------- 8. Model Training and Validation ----------------------
# 1. Split into training and validation sets (stratified sampling)
print("\n===== Preparing Training Data =====")
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(df, df['label']))
train_df = df.iloc[train_idx]
val_df = df.iloc[val_idx]

# 2. Create datasets and data loaders
train_dataset = SentimentDataset(train_df, word2idx, sent_dict, MAX_LEN)
val_dataset = SentimentDataset(val_df, word2idx, sent_dict, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 3. Initialize model and training configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentimentLSTM(
    vocab_size=len(word2idx),
    embed_dim=EMBED_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=3  # Three categories: neutral, negative, positive
).to(device)

# Load pre-trained word embeddings (optional)
embedding_matrix = np.zeros((len(word2idx), EMBED_DIM))
for word, idx in word2idx.items():
    if word in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word]
model.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Reduce learning rate

# 4. Training loop (with early stopping)
print("\n===== Starting Training =====")
best_val_acc = 0.0
no_improvement = 0

for epoch in range(EPOCHS):
    # Training phase
    model.train()
    total_train_loss = 0
    with tqdm(train_loader, unit='batch') as tepoch:
        for batch in tepoch:
            tepoch.set_description(f"Epoch {epoch+1}")
            
            # Load data to device
            input_ids = batch['input_ids'].to(device)
            pos_count = batch['pos_count'].to(device)
            neg_weight = batch['neg_weight'].to(device)
            labels = batch['label'].to(device)
            
            # Forward propagation + backward optimization
            optimizer.zero_grad()
            outputs = model({'input_ids': input_ids, 'pos_count': pos_count, 'neg_weight': neg_weight})
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
            tepoch.set_postfix(loss=loss.item())

    # Validation phase
    model.eval()
    total_val_correct = 0
    total_val_samples = 0
    val_preds = []
    val_labels = []
    
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            pos_count = batch['pos_count'].to(device)
            neg_weight = batch['neg_weight'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model({'input_ids': input_ids, 'pos_count': pos_count, 'neg_weight': neg_weight})
            preds = torch.argmax(outputs, dim=1)
            
            total_val_correct += (preds == labels).sum().item()
            total_val_samples += labels.size(0)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    # Calculate accuracy
    val_acc = total_val_correct / total_val_samples
    train_loss = total_train_loss / len(train_loader)
    
    # Calculate precision, recall, F1 score
    if total_val_samples > 0:
        precision, recall, f1, _ = precision_recall_fscore_support(
            val_labels, val_preds, average=None, labels=[0, 1, 2]
        )
        # Calculate micro-average and macro-average
        precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
            val_labels, val_preds, average='micro', labels=[0, 1, 2]
        )
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
            val_labels, val_preds, average='macro', labels=[0, 1, 2]
        )
        
        # Print metrics
        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}")
        print(f"  Per-class metrics (0: neutral, 1: negative, 2: positive):")
        print(f"  Precision: {precision[0]:.4f}, {precision[1]:.4f}, {precision[2]:.4f}")
        print(f"  Recall: {recall[0]:.4f}, {recall[1]:.4f}, {recall[2]:.4f}")
        print(f"  F1-score: {f1[0]:.4f}, {f1[1]:.4f}, {f1[2]:.4f}")
        print(f"  Micro-average: Precision={precision_micro:.4f}, Recall={recall_micro:.4f}, F1={f1_micro:.4f}")
        print(f"  Macro-average: Precision={precision_macro:.4f}, Recall={recall_macro:.4f}, F1={f1_macro:.4f}")
    else:
        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}")
        print("  Validation set has 0 samples, cannot calculate classification metrics")

    # Early stopping check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        no_improvement = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"Model saved (validation accuracy improved to {val_acc:.4f})")
    else:
        no_improvement += 1
        if no_improvement >= EARLY_STOPPING_PATIENCE:
            print(f"Early stopping triggered: validation accuracy did not improve for {EARLY_STOPPING_PATIENCE} consecutive epochs")
            break

print(f"\nTraining completed, best validation accuracy: {best_val_acc:.4f}")


# ---------------------- 9. Sentiment Prediction and Result Output ----------------------
def predict_sentiment(titles, model, word2idx, sent_dict, max_len, device):
    """Predict sentiment for new titles"""
    model.eval()
    scores = []
    with torch.no_grad():
        for title in tqdm(titles, desc="Predicting"):
            # Preprocessing
            tokens = preprocess_text(title, stopwords)
            input_ids = [word2idx.get(word, 0) for word in tokens]
            input_ids = input_ids[:max_len] + [0]*(max_len - len(input_ids)) if len(input_ids) < max_len else input_ids[:max_len]
            input_ids = torch.LongTensor([input_ids]).to(device)
            
            # Calculate sentiment features
            pos_count = torch.FloatTensor([[sum(1 for word in tokens if sent_dict.get(word, (None,))[0] == 'positive')]]).to(device)
            neg_weight = torch.FloatTensor([[sum(sent_dict[word][1] for word in tokens if sent_dict.get(word, (None,))[0] == 'negative')]]).to(device)
            
            # Model prediction
            outputs = model({'input_ids': input_ids, 'pos_count': pos_count, 'neg_weight': neg_weight})
            probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]
            scores.append(probs[2] - probs[1])  # Positive probability - negative probability
    return scores

# Predict sentiment for original data
df['score'] = predict_sentiment(df['标题'].tolist(), model, word2idx, sent_dict, MAX_LEN, device)

# Keep 4 decimal places
df['score'] = df['score'].round(4)  

# Merge with futures data
print("\n===== Merging Futures Data =====")
# Read futures data
futures_data = pd.read_csv(f'data/{KEYWORD}.csv', index_col=0)  
# Extract date and convert to string
futures_data['date'] = pd.to_datetime(futures_data['time']).dt.date.astype(str)
futures_data = futures_data.drop(columns=['time'])  # Remove original time column
# Calculate daily sentiment scores
daily_score = df.groupby(pd.to_datetime(df['最后更新时间']).dt.date.astype(str))['score'].mean().reset_index()
daily_score.columns = ['date', 'score']
# Merge data and adjust column order
final_data = futures_data.merge(daily_score, on='date', how='left').fillna(0)
# Adjust column order
target_columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'score']
# target_columns = ['date', 'code', 'open', 'high', 'low', 'close', 'volume', 'amount', 'open_interest', 'score']
final_data = final_data[target_columns]
# Save results
final_data.to_csv(OUTPUT_PATH, index=False)
print(f"✅ Final results saved to {OUTPUT_PATH}")

# Financial Time Series Prediction with DQN-Enhanced Hybrid Transformer and Sentiment Features (DQN-HTS-EF)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from scipy import stats
import matplotlib.pyplot as plt
import random
from collections import deque
from datetime import datetime

# ---------------------- 1. 基础配置与数据加载 ----------------------
# 设备配置（GPU优先）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 数据参数配置（需确保data文件夹存在）
KEYWORD = "CSI100"  # 替换为你的数据关键词（如股票名称）
data_path = f'data/{KEYWORD}_data_with_sentiment_scores.csv'
time_steps = 7  # 时间序列窗口长度（用前7天预测后1天）
y_col = 1  # 预测目标：收盘价（对应net_df中'close'列的索引）

# 读取与预处理数据
data = pd.read_csv(data_path)
dates = pd.to_datetime(data['date'])  # 日期列（用于后续可视化）
net_df = data[['open', 'close', 'high', 'low', 'volume', 'score']]  # 特征列（含情感分数）

# 缺失值填充（前向填充）与标准化（Min-Max到[0,1]）
data.fillna(method='ffill', inplace=True)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(net_df)
features = scaled_data  # 标准化后的特征矩阵


# ---------------------- 2. 时间序列数据集构造 ----------------------
def create_dataset(X, y_col, time_steps):
    """
    构造时间序列训练/测试集
    参数：
        X: 标准化后的特征矩阵 (n_samples, n_features)
        y_col: 目标列索引（收盘价）
        time_steps: 时间窗口长度
    返回：
        Xs: 输入特征 (n_samples-time_steps, time_steps, n_features)
        ys: 目标值 (n_samples-time_steps,)
    """
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])  # 截取前time_steps步作为输入
        ys.append(X[i + time_steps, y_col])  # 第time_steps+1步的收盘价作为目标
    return np.array(Xs), np.array(ys)

# 生成时间序列数据
X, y = create_dataset(features, y_col, time_steps)

# 数据集拆分（训练集22.2%，测试集77.8%，即2:7比例）
test_ratio = 7 / 9
test_size = int(len(X) * test_ratio)
X_train, X_test = X[:-test_size], X[-test_size:]  # 输入特征拆分
y_train, y_test = y[:-test_size], y[-test_size:]  # 目标值拆分

# 提取测试集对应的日期（用于后续可视化和结果保存）
test_dates = dates[time_steps + len(X_train):time_steps + len(X_train) + len(X_test)]
test_dates = test_dates.dt.strftime("%Y-%m-%d")

# 转换为PyTorch张量（适配Transformer模型）
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)


# ---------------------- 3. 模型定义（Transformer + SVR） ----------------------
# 3.1 Transformer基础模块（多头注意力+前馈网络）
class TransformerBlock(nn.Module):
    def __init__(self, input_dim, num_heads, ff_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.self_attn = nn.MultiheadAttention(input_dim, num_heads, batch_first=True)  # 多头自注意力
        self.feed_forward = nn.Sequential(  # 前馈网络
            nn.Linear(input_dim, ff_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, input_dim)
        )
        self.norm1 = nn.LayerNorm(input_dim)  # 层归一化1
        self.norm2 = nn.LayerNorm(input_dim)  # 层归一化2
        self.dropout = nn.Dropout(dropout)  #  dropout正则化

    def forward(self, x):
        # 自注意力层
        attn_output, _ = self.self_attn(x, x, x)  # (batch_size, time_steps, input_dim)
        x = self.norm1(x + self.dropout(attn_output))  # 残差连接+归一化
        # 前馈网络层
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))  # 残差连接+归一化
        return x

# 3.2 单一层Transformer模型
class SingleTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, ff_dim, dropout=0.1):
        super(SingleTransformer, self).__init__()
        self.transformer_block = TransformerBlock(input_dim, num_heads, ff_dim, dropout)
        self.fc = nn.Linear(input_dim, 1)  # 输出层（预测收盘价）

    def forward(self, x):
        x = self.transformer_block(x)  # (batch_size, time_steps, input_dim)
        x = x[:, -1, :]  # 取最后一个时间步的特征（用于预测）
        return self.fc(x).squeeze(-1)  # 压缩维度（batch_size,）

# 3.3 多层Transformer模型（2层）
class MultiLayerTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, ff_dim, num_layers=2, dropout=0.1):
        super(MultiLayerTransformer, self).__init__()
        self.layers = nn.ModuleList([  # 堆叠Transformer层
            TransformerBlock(input_dim, num_heads, ff_dim, dropout) 
            for _ in range(num_layers)
        ])
        self.fc = nn.Linear(input_dim, 1)  # 输出层

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)  # 逐层传递
        x = x[:, -1, :]  # 取最后一个时间步
        return self.fc(x).squeeze(-1)

# 3.4 双向Transformer模型（前向+后向特征融合）
class BidirectionalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, ff_dim, dropout=0.1):
        super(BidirectionalTransformer, self).__init__()
        self.forward_layer = TransformerBlock(input_dim, num_heads, ff_dim, dropout)  # 前向层
        self.backward_layer = TransformerBlock(input_dim, num_heads, ff_dim, dropout)  # 后向层
        self.fc = nn.Linear(2 * input_dim, 1)  # 融合前向+后向特征（维度翻倍）

    def forward(self, x):
        forward_out = self.forward_layer(x)  # 前向传播
        backward_out = self.backward_layer(x.flip(1))  # 后向传播（翻转时间步）
        # 融合最后一个时间步的前向+后向特征
        combined = torch.cat([forward_out[:, -1], backward_out[:, -1]], dim=-1)
        return self.fc(combined).squeeze(-1)

# 3.5 早停机制（防止过拟合，无验证集时自动跳过）
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = 5
        self.min_delta = 1e-3
        self.best_loss = None  # 最佳验证损失
        self.counter = 0  # 无改进计数器
        self.early_stop = False  # 早停标志

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss  # 初始化最佳损失
        elif val_loss >= self.best_loss - self.min_delta:
            self.counter += 1  # 无改进，计数器+1
            if self.counter >= self.patience:
                self.early_stop = True  # 触发早停
        else:
            self.best_loss = val_loss  # 更新最佳损失
            self.counter = 0  # 重置计数器


# ---------------------- 4. 模型训练（Transformer + SVR） ----------------------
# 4.1 Transformer训练函数
def train_model(model, X_train, y_train, X_val=None, y_val=None, epochs=200, lr=0.001, batch_size=32, patience=10):
    """
    训练Transformer模型（支持验证集早停）
    参数：
        model: 待训练的Transformer模型
        X_train/y_train: 训练集
        X_val/y_val: 验证集（可选）
        epochs: 最大训练轮次
        lr: 学习率
        batch_size: 批次大小
        patience: 早停等待次数
    返回：
        训练后的模型
    """
    criterion = nn.MSELoss()  # 损失函数（均方误差，适合回归任务）
    optimizer = optim.Adam(model.parameters(), lr=lr)  # 优化器（Adam）
    # 仅当有验证集时初始化早停
    early_stopping = EarlyStopping(patience=patience) if (X_val is not None and y_val is not None) else None

    for epoch in range(epochs):
        model.train()  # 训练模式（启用dropout）
        total_loss = 0  # 累计训练损失

        # 批次训练
        for i in range(0, len(X_train), batch_size):
            batch_x = X_train[i:i+batch_size]  # 批次输入
            batch_y = y_train[i:i+batch_size]  # 批次目标

            # 梯度清零→前向传播→计算损失→反向传播→参数更新
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()  # 累加损失

        # 验证集早停（仅当有验证集时执行）
        if early_stopping is not None and X_val is not None and y_val is not None:
            model.eval()  # 评估模式（关闭dropout）
            with torch.no_grad():  # 禁用梯度计算（节省内存）
                val_outputs = model(X_val)
                val_loss = criterion(val_outputs, y_val).item()

            early_stopping(val_loss)
            if early_stopping.early_stop:
                print(f"Early stopping at epoch {epoch+1} (val_loss: {val_loss:.6f})")
                break

    return model

# 4.2 初始化并训练3个Transformer模型
# 模型超参数（经网格搜索优化后的最佳参数）
single_best = {'num_heads': 2, 'ff_dim': 32}  # 单一层Transformer参数
multi_best = {'num_heads': 2, 'ff_dim': 64}   # 多层Transformer参数
bi_best = {'num_heads': 2, 'ff_dim': 32}      # 双向Transformer参数
input_dim = features.shape[1]  # 输入特征维度（6：open/close/high/low/volume/score）

# 初始化模型（部署到指定设备）
single_transformer = SingleTransformer(input_dim, **single_best).to(device)
multi_layer_transformer = MultiLayerTransformer(input_dim, **multi_best, num_layers=2).to(device)
bidirectional_transformer = BidirectionalTransformer(input_dim, **bi_best).to(device)

# 训练模型（无验证集，早停自动跳过）
print("\nTraining Single-layer Transformer...")
single_transformer = train_model(single_transformer, X_train, y_train, X_val=None, y_val=None, patience=15)

print("Training Multi-layer Transformer...")
multi_layer_transformer = train_model(multi_layer_transformer, X_train, y_train, X_val=None, y_val=None, patience=15)

print("Training Bidirectional Transformer...")
bidirectional_transformer = train_model(bidirectional_transformer, X_train, y_train, X_val=None, y_val=None, patience=15)

# 4.3 训练SVR模型（传统机器学习基线）
# SVR输入需展平（时间序列窗口→1维特征）
X_train_svr = X_train.cpu().numpy().reshape(-1, time_steps * input_dim)  # (n_samples, time_steps*input_dim)
y_train_svr = y_train.cpu().numpy()  # 目标值转回numpy

# 网格搜索优化SVR超参数（C：正则化强度，gamma：RBF核带宽）
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.001, 0.01]}
grid = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=3, n_jobs=-1)  # 3折交叉验证，多线程加速
grid.fit(X_train_svr, y_train_svr)
svr = grid.best_estimator_  # 最佳SVR模型
print(f"Best SVR params: {grid.best_params_}")


# ---------------------- 5. 模型预测与损失计算（MCS输入准备） ----------------------
def get_predictions(model, X_test, time_steps, input_dim):
    """
    获取模型预测结果（适配Transformer和SVR）
    参数：
        model: 训练后的模型（Transformer或SVR）
        X_test: 测试集输入
        time_steps: 时间窗口长度
        input_dim: 输入特征维度
    返回：
        预测结果（numpy数组）
    """
    if isinstance(model, torch.nn.Module):  # Transformer模型（PyTorch）
        model.eval()
        with torch.no_grad():
            return model(X_test).cpu().numpy()  # 转回CPU并转为numpy
    else:  # SVR模型（sklearn）
        X_test_flat = X_test.cpu().numpy().reshape(-1, time_steps * input_dim)  # 展平输入
        return model.predict(X_test_flat)

# 5.1 获取所有模型的测试集预测（标准化后）
single_pred = get_predictions(single_transformer, X_test, time_steps, input_dim)
multi_pred = get_predictions(multi_layer_transformer, X_test, time_steps, input_dim)
bi_pred = get_predictions(bidirectional_transformer, X_test, time_steps, input_dim)
svr_pred = get_predictions(svr, X_test, time_steps, input_dim)

# 5.2 逆标准化（将预测值/真实值转回原始价格尺度）
def inverse_transform_pred(pred, scaler, y_col, input_dim):
    """
    逆标准化函数（Min-Max逆变换）
    参数：
        pred: 标准化后的预测值/真实值
        scaler: 训练好的MinMaxScaler
        y_col: 目标列索引
        input_dim: 输入特征维度
    返回：
        原始尺度的值
    """
    dummy = np.zeros((len(pred), input_dim))  # 构造与特征矩阵同维度的虚拟矩阵
    dummy[:, y_col] = pred  # 仅目标列填入值，其他列填0
    return scaler.inverse_transform(dummy)[:, y_col]  # 逆变换后取目标列

# 逆标准化所有结果（原始价格尺度）
y_test_inverse = inverse_transform_pred(y_test.cpu().numpy(), scaler, y_col, input_dim)
single_pred_inverse = inverse_transform_pred(single_pred, scaler, y_col, input_dim)
multi_pred_inverse = inverse_transform_pred(multi_pred, scaler, y_col, input_dim)
bi_pred_inverse = inverse_transform_pred(bi_pred, scaler, y_col, input_dim)
svr_pred_inverse = inverse_transform_pred(svr_pred, scaler, y_col, input_dim)

# 5.3 计算每个时间点的模型损失（MAE：平均绝对误差，对异常值更稳健）
def calculate_timewise_loss(y_true, y_pred, loss_type='mae'):
    """
    计算每个时间点的损失（逐样本损失）
    参数：
        y_true: 真实值（原始尺度）
        y_pred: 预测值（原始尺度）
        loss_type: 损失类型（'mae' 或 'mse'）
    返回：
        逐时间点损失数组（长度=测试集样本数）
    """
    if loss_type == 'mae':
        return np.abs(y_true - y_pred)  # 逐点MAE损失
    elif loss_type == 'mse':
        return (y_true - y_pred) ** 2  # 逐点MSE损失
    else:
        raise ValueError("loss_type must be 'mae' or 'mse'")

# 定义模型名称与预测结果映射（便于后续迭代）
models = {
    'Single-Transformer': single_pred_inverse,
    'Multi-Transformer': multi_pred_inverse,
    'Bi-Transformer': bi_pred_inverse,
    'SVR': svr_pred_inverse
}

# 计算每个模型的逐时间点损失（选用MAE，对异常值更稳健）
timewise_loss = {}
for name, pred in models.items():
    timewise_loss[name] = calculate_timewise_loss(y_test_inverse, pred, loss_type='mae')

# 将损失转换为矩阵格式（行=时间点，列=模型）
loss_matrix = np.column_stack([timewise_loss[name] for name in models.keys()])
model_names = list(models.keys())  # 模型名称列表（与loss_matrix列顺序对应）


# ---------------------- 6. MCS检验（模型信度集筛选） ----------------------
def bootstrap_pvalue(loss_a, loss_b, n_bootstrap=1000, alpha=0.05):
    """
    自助法（Bootstrap）计算成对模型损失差异的p值
    参数：
        loss_a: 模型A的逐时间点损失
        loss_b: 模型B的逐时间点损失
        n_bootstrap: 自助抽样次数（越大越稳健，建议≥1000）
        alpha: 显著性水平（默认0.05）
    返回：
        p值: 原假设（A与B损失无差异）的概率
        is_significant: 是否存在显著差异（p < alpha）
    """
    n_samples = len(loss_a)
    diff_obs = loss_a - loss_b  # 观测到的损失差异（A - B）
    null_distribution = []  # 自助法生成的零假设分布

    # 自助抽样：生成n_bootstrap个重采样样本
    for _ in range(n_bootstrap):
        # 有放回抽样（索引）
        idx = np.random.choice(n_samples, size=n_samples, replace=True)
        diff_boot = diff_obs[idx]  # 重采样的损失差异
        null_distribution.append(np.mean(diff_boot))  # 计算重采样差异的均值

    # 计算p值：零假设下，|重采样均值| ≥ |观测均值| 的比例
    obs_mean = np.mean(diff_obs)
    p_value = np.sum(np.abs(null_distribution) >= np.abs(obs_mean)) / n_bootstrap
    return p_value, (p_value < alpha)

def run_mcs(loss_matrix, model_names, n_bootstrap=1000, alpha=0.05):
    """
    运行MCS检验，逐步剔除最差模型，获取模型信度集（MCS）
    参数：
        loss_matrix: 损失矩阵（行=时间点，列=模型）
        model_names: 模型名称列表（与列顺序对应）
        n_bootstrap: 自助抽样次数
        alpha: 显著性水平
    返回：
        mcs_set: 模型信度集（最终无显著差异的模型）
        elimination_order: 模型剔除顺序（从最差到次差）
        p_value_history: 每次检验的p值记录
    """
    # 初始化MCS候选集（包含所有模型）
    current_candidates = list(range(len(model_names)))  # 用索引代表模型
    elimination_order = []  # 记录剔除顺序（模型名称）
    p_value_history = []    # 记录每次检验的p值

    # 逐步剔除：直到候选集内无显著差异
    while len(current_candidates) > 1:
        n_candidates = len(current_candidates)
        p_matrix = np.ones((n_candidates, n_candidates))  # p值矩阵（i,j：模型i vs 模型j）

        # 计算所有成对模型的p值
        for i in range(n_candidates):
            for j in range(i+1, n_candidates):
                # 获取两个模型的损失
                loss_i = loss_matrix[:, current_candidates[i]]
                loss_j = loss_matrix[:, current_candidates[j]]
                # 计算p值
                p_val, _ = bootstrap_pvalue(loss_i, loss_j, n_bootstrap, alpha)
                p_matrix[i, j] = p_val
                p_matrix[j, i] = p_val  # 对称矩阵

        # 计算每个模型的"最小p值"（与其他所有模型比较的最小p值）
        min_p_per_model = np.min(p_matrix, axis=1)
        # 找到"最小p值最小"的模型（表现最差，最可能被剔除）
        worst_idx_in_curr = np.argmin(min_p_per_model)
        worst_model_idx = current_candidates[worst_idx_in_curr]
        worst_model_name = model_names[worst_model_idx]
        worst_p = min_p_per_model[worst_idx_in_curr]

        # 记录结果
        elimination_order.append(worst_model_name)
        p_value_history.append(worst_p)

        # 剔除最差模型
        current_candidates.pop(worst_idx_in_curr)
        print(f"剔除模型：{worst_model_name}，p值：{worst_p:.4f}（<{alpha}，显著差异）")

    # 最终候选集即为模型信度集（MCS）
    mcs_set = [model_names[idx] for idx in current_candidates]
    print(f"\nMCS模型信度集（无显著差异的最佳模型）：{mcs_set}")
    return mcs_set, elimination_order, p_value_history

# 运行MCS检验（自助抽样1000次，显著性水平0.05）
print("\n=== 开始MCS检验（逐步剔除最差模型） ===")
mcs_set, elimination_order, p_value_history = run_mcs(
    loss_matrix=loss_matrix,
    model_names=model_names,
    n_bootstrap=1000,
    alpha=0.05
)


# ---------------------- 7. 基于MCS信度集的组合预测 ----------------------
def mcs_ensemble_pred(mcs_set, models, weight_type='equal'):
    """
    基于MCS信度集的组合预测（等权重或损失反比权重）
    参数：
        mcs_set: MCS模型信度集（模型名称列表）
        models: 所有模型的预测结果（{模型名: 预测数组}）
        weight_type: 权重类型（'equal'：等权重，'inv_loss'：损失反比权重）
    返回：
        ensemble_pred: 组合预测结果
        weights: 各模型的权重
    """
    # 提取MCS集中模型的预测和损失
    mcs_preds = [models[name] for name in mcs_set]
    mcs_losses = [timewise_loss[name] for name in mcs_set]

    if weight_type == 'equal':
        # 等权重：每个模型权重=1/模型数量
        n_mcs = len(mcs_set)
        weights = np.ones(n_mcs) / n_mcs
    elif weight_type == 'inv_loss':
        # 损失反比权重：损失越小，权重越大（基于平均损失）
        mean_losses = [np.mean(loss) for loss in mcs_losses]
        inv_loss = 1 / np.array(mean_losses)  # 损失反比
        weights = inv_loss / np.sum(inv_loss)  # 归一化
    else:
        raise ValueError("weight_type must be 'equal' or 'inv_loss'")

    # 计算组合预测（加权求和）
    ensemble_pred = np.average(mcs_preds, weights=weights, axis=0)
    # 打印权重信息
    print(f"\nMCS组合预测权重（{weight_type}）：")
    for name, w in zip(mcs_set, weights):
        print(f"  {name}: {w:.4f}")
    return ensemble_pred, weights

# 生成MCS组合预测（默认等权重，可切换为'inv_loss'）
mcs_ensemble_pred, mcs_weights = mcs_ensemble_pred(
    mcs_set=mcs_set,
    models=models,
    weight_type='equal'
)


# ---------------------- 8. 结果评估与可视化 ----------------------
# 8.1 计算所有模型（含MCS组合）的评估指标
def calculate_metrics(y_true, y_pred, name):
    """计算回归任务的核心指标：RMSE、MAE、R²、MAPE"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    # MAPE（避免除以0，用掩码处理）
    mask = y_true != 0
    if np.sum(mask) == 0:
        mape = float('nan')
    else:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    return {
        'Model': name,
        'MSE': mse,        
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE(%)': mape
    }

# 计算所有模型的指标
metrics_list = []
# 原始4个模型
for name, pred in models.items():
    metrics_list.append(calculate_metrics(y_test_inverse, pred, name))
# MCS组合模型
metrics_list.append(calculate_metrics(y_test_inverse, mcs_ensemble_pred, 'MCS-Ensemble'))

# 转换为DataFrame便于查看
metrics_df = pd.DataFrame(metrics_list)
print("\n=== 所有模型（含MCS组合）评估指标 ===")
print(metrics_df.round(4))

# 8.2 可视化：真实值 vs 各模型预测值（含MCS组合）
plt.figure(figsize=(14, 8))
# 真实值（黑色实线，突出显示）
plt.plot(test_dates, y_test_inverse, '-', color='black', linewidth=2.5, label='Actual Price', alpha=0.9)
# 原始4个模型（虚线）
colors = ['tab:blue', 'tab:green', 'tab:red', 'tab:orange']
styles = ['--', '--', '--', '-.']
for i, (name, pred) in enumerate(models.items()):
    plt.plot(
        test_dates, pred, styles[i], color=colors[i], 
        linewidth=1.5, label=name, alpha=0.7
    )
# MCS组合模型（紫色实线，加粗）
plt.plot(
    test_dates, mcs_ensemble_pred, '-', color='tab:purple', 
    linewidth=2.5, label='MCS-Ensemble', alpha=0.9
)

# 日期轴处理（避免过于密集，按月份显示）
unique_months = []
unique_indices = []
month_set = set()
for idx, date in enumerate(pd.to_datetime(test_dates)):
    month_key = date.strftime('%Y-%m')
    if month_key not in month_set:
        unique_months.append(month_key)
        unique_indices.append(idx)
        month_set.add(month_key)
plt.xticks(unique_indices, unique_months, rotation=45)

# 图表标签与图例
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price', fontsize=12)
plt.title(f'Price Prediction Comparison (Corn) - MCS Ensemble', fontsize=14)
plt.legend(fontsize=10, loc='best')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


# ---------------------- 9. 结果保存（便于后续分析） ----------------------
# 9.1 保存所有预测结果（含MCS组合）
prediction_data = {
    'Date': test_dates,
    'Actual_Price': y_test_inverse,
    'Single-Transformer': single_pred_inverse,
    'Multi-Transformer': multi_pred_inverse,
    'Bi-Transformer': bi_pred_inverse,
    'SVR': svr_pred_inverse,
    'MCS-Ensemble': mcs_ensemble_pred
}
prediction_df = pd.DataFrame(prediction_data)
prediction_df.to_csv(f'data/{KEYWORD}_mcs_prediction_results.csv', index=False)
print(f"\n预测结果已保存至：data/{KEYWORD}_mcs_prediction_results.csv")

# 9.2 保存MCS检验结果
mcs_result_data = {
    'MCS_Set': [', '.join(mcs_set)],
    'Elimination_Order': [', '.join(elimination_order)],
    'Elimination_P_Values': [', '.join([f'{p:.4f}' for p in p_value_history])],
    'MCS_Weights': [', '.join([f'{name}:{w:.4f}' for name, w in zip(mcs_set, mcs_weights)])]
}
mcs_result_df = pd.DataFrame(mcs_result_data)
mcs_result_df.to_csv(f'data/{KEYWORD}_mcs_test_results.csv', index=False)
print(f"MCS检验结果已保存至：data/{KEYWORD}_mcs_test_results.csv")

# 9.3 保存评估指标
metrics_df.to_csv(f'data/{KEYWORD}_model_metrics.csv', index=False)
print(f"模型评估指标已保存至：data/{KEYWORD}_model_metrics.csv")

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from scipy import stats
import matplotlib.pyplot as plt
import math
import random
from collections import deque
from datetime import datetime

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 需先定义KEYWORD（示例：CSI100，根据你的数据修改）
KEYWORD = "Corn"

# Read data
data = pd.read_csv(f'data/{KEYWORD}_data_with_sentiment_scores.csv')
dates = pd.to_datetime(data['date'])
net_df = data[['open', 'close', 'high', 'low', 'volume', 'score']]

# Data preprocessing
data.fillna(method='ffill', inplace=True)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(net_df)
features = scaled_data
y_col = 1  # 预测目标：收盘价（close）

# Create time series dataset（不变）
def create_dataset(X, y_col, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(X[i + time_steps, y_col])
    return np.array(Xs), np.array(ys)

time_steps = 7
X, y = create_dataset(features, y_col, time_steps)

# ★ 1. 调整训练集/测试集比例为2:7（训练集22.2%，测试集77.8%）
test_ratio = 7 / 9  # 2:7 → 测试集占7份
test_size = int(len(X) * test_ratio)
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]

# Extract test set dates（不变）
test_dates = dates[time_steps + len(X_train):time_steps + len(X_train) + len(X_test)]
test_dates = test_dates.dt.strftime("%Y-%m-%d")

# Convert to PyTorch tensors（不变）
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# ---------------------- Transformer模型定义（全部不变）----------------------
class TransformerBlock(nn.Module):
    def __init__(self, input_dim, num_heads, ff_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.self_attn = nn.MultiheadAttention(input_dim, num_heads, batch_first=True)
        self.feed_forward = nn.Sequential(
            nn.Linear(input_dim, ff_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, input_dim)
        )
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.self_attn(x, x, x)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

class SingleTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, ff_dim, dropout=0.1):
        super(SingleTransformer, self).__init__()
        self.transformer_block = TransformerBlock(input_dim, num_heads, ff_dim, dropout)
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        x = self.transformer_block(x)
        x = x[:, -1, :]
        return self.fc(x).squeeze(-1)

class MultiLayerTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, ff_dim, num_layers=2, dropout=0.1):
        super(MultiLayerTransformer, self).__init__()
        self.layers = nn.ModuleList([
            TransformerBlock(input_dim, num_heads, ff_dim, dropout) 
            for _ in range(num_layers)
        ])
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x[:, -1, :]
        return self.fc(x).squeeze(-1)

class BidirectionalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, ff_dim, dropout=0.1):
        super(BidirectionalTransformer, self).__init__()
        self.forward_layer = TransformerBlock(input_dim, num_heads, ff_dim, dropout)
        self.backward_layer = TransformerBlock(input_dim, num_heads, ff_dim, dropout)
        self.fc = nn.Linear(2*input_dim, 1)

    def forward(self, x):
        forward_out = self.forward_layer(x)
        backward_out = self.backward_layer(x.flip(1))
        combined = torch.cat([forward_out[:, -1], backward_out[:, -1]], dim=-1)
        return self.fc(combined).squeeze(-1)

# Early stopping机制
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss >= self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# ★ 2. 修改训练函数
def train_model(model, X_train, y_train, X_val=None, y_val=None, epochs=200, lr=0.001, batch_size=32, patience=10):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    early_stopping = EarlyStopping(patience=patience) if (X_val is not None and y_val is not None) else None
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for i in range(0, len(X_train), batch_size):
            batch_x = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # 无验证集，跳过验证和早停
        if early_stopping is not None and X_val is not None and y_val is not None:
            model.eval()
            with torch.no_grad():
                val_outputs = model(X_val)
                val_loss = criterion(val_outputs, y_val).item()
            
            early_stopping(val_loss)
            if early_stopping.early_stop:
                break
    
    return model


X_train_split, y_train_split = X_train, y_train
X_val, y_val = None, None

# Model parameters
single_best = {'num_heads': 2, 'ff_dim': 32}
multi_best = {'num_heads': 2, 'ff_dim': 64}
bi_best = {'num_heads': 2, 'ff_dim': 32}

# 初始化并训练Transformer模型
single_transformer = SingleTransformer(features.shape[1], **single_best).to(device)
multi_layer_transformer = MultiLayerTransformer(features.shape[1],** multi_best, num_layers=2).to(device)
bidirectional_transformer = BidirectionalTransformer(features.shape[1], **bi_best).to(device)

single_transformer = train_model(single_transformer, X_train_split, y_train_split, X_val, y_val, patience=15)
multi_layer_transformer = train_model(multi_layer_transformer, X_train_split, y_train_split, X_val, y_val, patience=15)
bidirectional_transformer = train_model(bidirectional_transformer, X_train_split, y_train_split, X_val, y_val, patience=15)

# 训练SVR模型
X_train_svr = X_train.cpu().numpy().reshape(-1, time_steps * features.shape[1])
y_train_svr = y_train.cpu().numpy()

param_grid = {'C': [0.1, 1, 10], 'gamma': [0.001, 0.01]}
grid = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=3)
grid.fit(X_train_svr, y_train_svr)
svr = grid.best_estimator_

# 获取各模型预测结果
def get_predictions(model, X_test):
    if isinstance(model, torch.nn.Module):
        model.eval()
        with torch.no_grad():
            return model(X_test).cpu().numpy()
    else:
        X_test_flat = X_test.cpu().numpy().reshape(-1, time_steps * features.shape[1])
        return model.predict(X_test_flat)

single_pred = get_predictions(single_transformer, X_test)
multi_pred = get_predictions(multi_layer_transformer, X_test)
bi_pred = get_predictions(bidirectional_transformer, X_test)
svr_pred = get_predictions(svr, X_test)

# 计算MAPE
def calculate_mape(y_true, y_pred):
    mask = y_true != 0
    if np.sum(mask) == 0:
        return float('nan')
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# ---------------------- DQN整合----------------------
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()

    def _build_model(self):
        model = nn.Sequential(
            nn.Linear(self.state_size, 24),
            nn.ReLU(),
            nn.Linear(24, 24),
            nn.ReLU(),
            nn.Linear(24, self.action_size)
        )
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model(torch.tensor(state, dtype=torch.float32))
        action = np.argmax(act_values.detach().numpy())
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = np.array(state).reshape(1, self.state_size)
            next_state = np.array(next_state).reshape(1, self.state_size)
            
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model(torch.tensor(next_state, dtype=torch.float32)).detach().numpy()))
            target_f = self.model(torch.tensor(state, dtype=torch.float32))
            target_f[0][action] = target

# 训练DQN Agent
state_size = 4
action_size = 4
agent = DQNAgent(state_size, action_size)
batch_size = 32
episodes = 300

for e in range(episodes):
    state = np.array([svr_pred[0], single_pred[0], multi_pred[0], bi_pred[0]]).reshape(1, state_size)
    for t in range(len(y_test) - 1):
        action = agent.act(state)
        pred = [svr_pred[t], single_pred[t], multi_pred[t], bi_pred[t]][action]
        y_test_value = y_test[t].item()
        reward = -np.abs(pred - y_test_value)
        next_state = np.array([svr_pred[t + 1], single_pred[t + 1], multi_pred[t + 1], bi_pred[t + 1]]).reshape(1, state_size)
        done = t == len(y_test) - 2
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

# RL整合模型
class RLAgent(nn.Module):
    def __init__(self, num_models):
        super(RLAgent, self).__init__()
        self.fc1 = nn.Linear(num_models, 16)
        self.fc2 = nn.Linear(16, num_models)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=1)
        return x

# 训练整合模型
num_models = 4
agent_rl = RLAgent(num_models)  # 重命名为agent_rl，避免与DQNAgent冲突
optimizer = optim.Adam(agent_rl.parameters(), lr=0.001)
criterion = nn.MSELoss()

epochs = 500
all_preds = np.stack([single_pred, multi_pred, bi_pred, svr_pred], axis=1)
all_preds_tensor = torch.tensor(all_preds, dtype=torch.float32)

for epoch in range(epochs):
    actions = agent_rl(all_preds_tensor)
    selected_preds = (actions * all_preds_tensor).sum(dim=1)
    loss = criterion(selected_preds, y_test.cpu())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 生成最终DQN预测
with torch.no_grad():
    actions = agent_rl(all_preds_tensor)
    selected_preds = (actions * all_preds_tensor).sum(dim=1).numpy()

# 逆标准化
def inverse_transform_pred(pred):
    dummy = np.zeros((len(pred), features.shape[1]))
    dummy[:, y_col] = pred
    return scaler.inverse_transform(dummy)[:, y_col]

y_test_inverse = inverse_transform_pred(y_test.cpu().numpy())
single_pred_inverse = inverse_transform_pred(single_pred)
multi_pred_inverse = inverse_transform_pred(multi_pred)
bi_pred_inverse = inverse_transform_pred(bi_pred)
svr_pred_inverse = inverse_transform_pred(svr_pred)
dqn_pred_inverse = inverse_transform_pred(selected_preds) 

# DQN结果保存
dqn_data = {
    'date': test_dates,
    'DQN_Prediction': dqn_pred_inverse,
    'actual_price': y_test_inverse
}
dqn_prediction_df = pd.DataFrame(dqn_data)
dqn_prediction_df['date'] = pd.to_datetime(dqn_prediction_df['date']).dt.strftime("%Y-%m-%d")
dqn_file_name = f"data/{KEYWORD}_agent_integration_prediction_results.csv"
dqn_prediction_df.to_csv(dqn_file_name, index=False)

# 评估指标计算
rmse_dqn = np.sqrt(mean_squared_error(y_test_inverse, dqn_pred_inverse))
mse_dqn = mean_squared_error(y_test_inverse, dqn_pred_inverse)
r2_dqn = r2_score(y_test_inverse, dqn_pred_inverse)
mae_dqn = mean_absolute_error(y_test_inverse, dqn_pred_inverse)
mape_dqn = calculate_mape(y_test_inverse, dqn_pred_inverse)

rmse_svr = np.sqrt(mean_squared_error(y_test_inverse, svr_pred_inverse))
mse_svr = mean_squared_error(y_test_inverse, svr_pred_inverse)
r2_svr = r2_score(y_test_inverse, svr_pred_inverse)
mae_svr = mean_absolute_error(y_test_inverse, svr_pred_inverse)
mape_svr = calculate_mape(y_test_inverse, svr_pred_inverse)

rmse_single = np.sqrt(mean_squared_error(y_test_inverse, single_pred_inverse))
mse_single = mean_squared_error(y_test_inverse, single_pred_inverse)
r2_single = r2_score(y_test_inverse, single_pred_inverse)
mae_single = mean_absolute_error(y_test_inverse, single_pred_inverse)
mape_single = calculate_mape(y_test_inverse, single_pred_inverse)

rmse_multi = np.sqrt(mean_squared_error(y_test_inverse, multi_pred_inverse))
mse_multi = mean_squared_error(y_test_inverse, multi_pred_inverse)
r2_multi = r2_score(y_test_inverse, multi_pred_inverse)
mae_multi = mean_absolute_error(y_test_inverse, multi_pred_inverse)
mape_multi = calculate_mape(y_test_inverse, multi_pred_inverse)

rmse_bi = np.sqrt(mean_squared_error(y_test_inverse, bi_pred_inverse))
mse_bi = mean_squared_error(y_test_inverse, bi_pred_inverse)
r2_bi = r2_score(y_test_inverse, bi_pred_inverse)
mae_bi = mean_absolute_error(y_test_inverse, bi_pred_inverse)
mape_bi = calculate_mape(y_test_inverse, bi_pred_inverse)

# 原有指标输出
print("\n=== DQN及基础模型评估指标 ===")
print(f'Our DQN-HTS-EF - RMSE: {rmse_dqn:.4f}, MSE: {mse_dqn:.4f}, R^2: {r2_dqn:.4f}, MAE: {mae_dqn:.4f}, MAPE: {mape_dqn:.4f}%')
print(f'SVR Model - RMSE: {rmse_svr:.4f}, MSE: {mse_svr:.4f}, R^2: {r2_svr:.4f}, MAE: {mae_svr:.4f}, MAPE: {mape_svr:.4f}%')
print(f'Single-layer Transformer - RMSE: {rmse_single:.4f}, MSE: {mse_single:.4f}, R^2: {r2_single:.4f}, MAE: {mae_single:.4f}, MAPE: {mape_single:.4f}%')
print(f'Multi-layer Transformer - RMSE: {rmse_multi:.4f}, MSE: {mse_multi:.4f}, R^2: {r2_multi:.4f}, MAE: {mae_multi:.4f}, MAPE: {mape_multi:.4f}%')
print(f'Bidirectional Transformer - RMSE: {rmse_bi:.4f}, MSE: {mse_bi:.4f}, R^2: {r2_bi:.4f}, MAE: {mae_bi:.4f}, MAPE: {mape_bi:.4f}%')


# ---------------------- MCS集成 ----------------------
print("\n=== MCS集成计算 ===")
# 1. MCS核心函数
def calculate_timewise_loss(y_true, y_pred, loss_type='mae'):
    """计算逐时间点损失"""
    if loss_type == 'mae':
        return np.abs(y_true - y_pred)
    elif loss_type == 'mse':
        return (y_true - y_pred) ** 2
    raise ValueError("loss_type must be 'mae' or 'mse'")

def bootstrap_pvalue(loss_a, loss_b, n_bootstrap=1000, alpha=0.05):
    """自助法计算模型成对比较的p值"""
    n_samples = len(loss_a)
    diff_obs = loss_a - loss_b
    null_distribution = []
    for _ in range(n_bootstrap):
        idx = np.random.choice(n_samples, size=n_samples, replace=True)
        diff_boot = diff_obs[idx]
        null_distribution.append(np.mean(diff_boot))
    obs_mean = np.mean(diff_obs)
    p_value = np.sum(np.abs(null_distribution) >= np.abs(obs_mean)) / n_bootstrap
    return p_value, (p_value < alpha)

def run_mcs(loss_matrix, model_names, n_bootstrap=1000, alpha=0.05):
    """运行MCS检验，筛选模型信度集"""
    current_candidates = list(range(len(model_names)))
    elimination_order = []
    p_value_history = []

    while len(current_candidates) > 1:
        n_candidates = len(current_candidates)
        p_matrix = np.ones((n_candidates, n_candidates))
        
        # 计算所有成对模型的p值
        for i in range(n_candidates):
            for j in range(i+1, n_candidates):
                loss_i = loss_matrix[:, current_candidates[i]]
                loss_j = loss_matrix[:, current_candidates[j]]
                p_val, _ = bootstrap_pvalue(loss_i, loss_j, n_bootstrap, alpha)
                p_matrix[i, j] = p_val
                p_matrix[j, i] = p_val
        
        # 剔除最差模型（最小p值最小的模型）
        min_p_per_model = np.min(p_matrix, axis=1)
        worst_idx_in_curr = np.argmin(min_p_per_model)
        worst_model_idx = current_candidates[worst_idx_in_curr]
        worst_model_name = model_names[worst_model_idx]
        worst_p = min_p_per_model[worst_idx_in_curr]
        
        elimination_order.append(worst_model_name)
        p_value_history.append(worst_p)
        current_candidates.pop(worst_idx_in_curr)
        print(f"MCS剔除模型：{worst_model_name}，p值：{worst_p:.4f}（<{alpha}为显著差异）")

    # 最终信度集
    mcs_set = [model_names[idx] for idx in current_candidates]
    print(f"MCS模型信度集：{mcs_set}")
    return mcs_set, elimination_order, p_value_history

# 2. 基于原有基础模型预测构建MCS输入
models_mcs = {
    'Single-Transformer': single_pred_inverse,
    'Multi-Transformer': multi_pred_inverse,
    'Bi-Transformer': bi_pred_inverse,
    'SVR': svr_pred_inverse
}
model_names_mcs = list(models_mcs.keys())

# 3. 计算逐时间点损失矩阵
timewise_loss = {}
for name, pred in models_mcs.items():
    timewise_loss[name] = calculate_timewise_loss(y_test_inverse, pred, loss_type='mae')
loss_matrix = np.column_stack([timewise_loss[name] for name in model_names_mcs])

# 4. 运行MCS
mcs_set, elimination_order, p_value_history = run_mcs(
    loss_matrix=loss_matrix,
    model_names=model_names_mcs,
    n_bootstrap=1000,
    alpha=0.05
)

# 5. 生成MCS组合预测
def mcs_ensemble_pred(mcs_set, models):
    """MCS等权重组合预测"""
    mcs_preds = [models[name] for name in mcs_set]
    weights = np.ones(len(mcs_set)) / len(mcs_set)  # 等权重
    mcs_pred_inverse = np.average(mcs_preds, weights=weights, axis=0)
    # 打印MCS权重
    print(f"\nMCS组合权重（等权重）：")
    for name, w in zip(mcs_set, weights):
        print(f"  {name}: {w:.4f}")
    return mcs_pred_inverse, weights

mcs_pred_inverse, mcs_weights = mcs_ensemble_pred(mcs_set, models_mcs)

# 6. MCS评估指标
rmse_mcs = np.sqrt(mean_squared_error(y_test_inverse, mcs_pred_inverse))
mse_mcs = mean_squared_error(y_test_inverse, mcs_pred_inverse)
r2_mcs = r2_score(y_test_inverse, mcs_pred_inverse)
mae_mcs = mean_absolute_error(y_test_inverse, mcs_pred_inverse)
mape_mcs = calculate_mape(y_test_inverse, mcs_pred_inverse)

print(f"\nMCS集成模型评估指标：")
print(f'MCS-Ensemble - RMSE: {rmse_mcs:.4f}, MSE: {mse_mcs:.4f}, R^2: {r2_mcs:.4f}, MAE: {mae_mcs:.4f}, MAPE: {mape_mcs:.4f}%')


# ---------------------- Diebold-Mariano（DM）检验（MCS vs DQN） ----------------------
print("\n=== DM检验（MCS vs DQN） ===")
def diebold_mariano_test(y_true, pred_base, pred_comp, loss_type='mse'):
    """
    DM检验：对比基准模型（MCS）和对比模型（DQN）
    loss_type: 用MSE（与原有指标一致，关注大误差）
    """
    # 计算原始尺度损失
    if loss_type == 'mse':
        loss_base = (y_true - pred_base) ** 2  # MCS损失
        loss_comp = (y_true - pred_comp) ** 2  # DQN损失
    elif loss_type == 'mae':
        loss_base = np.abs(y_true - pred_base)
        loss_comp = np.abs(y_true - pred_comp)
    else:
        raise ValueError("loss_type仅支持'mse'或'mae'")
    
    # 损失差异（DQN - MCS）
    loss_diff = loss_comp - loss_base
    n = len(loss_diff)
    loss_diff_mean = np.mean(loss_diff)
    
    # Newey-West调整（处理时间序列自相关）
    gamma0 = np.var(loss_diff)  # 滞后0阶自协方差
    gamma1 = np.cov(loss_diff[:-1], loss_diff[1:])[0, 1]  # 滞后1阶自协方差
    se_dm = np.sqrt((gamma0 + 2 * gamma1) / n)  # 稳健标准误
    
    # DM统计量和p值
    dm_stat = loss_diff_mean / se_dm if se_dm != 0 else 0.0
    t_result = stats.ttest_1samp(loss_diff, 0)  # 双尾检验
    p_value = t_result.pvalue
    
    return dm_stat, p_value, loss_diff_mean

# 执行DM检验（MCS为基准，DQN为对比）
dm_stat, dm_p_value, dm_loss_diff_mean = diebold_mariano_test(
    y_true=y_test_inverse,
    pred_base=mcs_pred_inverse,  # 基准：MCS集成
    pred_comp=dqn_pred_inverse,  # 对比：原有DQN
    loss_type='mse'  # 与原有评估指标一致
)

# 打印DM结果
dm_result = pd.DataFrame({
    '基准模型': ['MCS-Ensemble'],
    '对比模型': ['DQN-HTS-EF'],
    'DM统计量': [round(dm_stat, 4)],
    'P值': [round(dm_p_value, 4)],
    '损失差异均值（DQN-MCS）': [round(dm_loss_diff_mean, 4)],
    '差异显著性（p<0.05）': ['是' if dm_p_value < 0.05 else '否'],
    '结论': [
        'MCS显著更优' if (dm_loss_diff_mean > 0 and dm_p_value < 0.05) else
        'DQN显著更优' if (dm_loss_diff_mean < 0 and dm_p_value < 0.05) else
        '无显著差异'
    ]
})
print("\nDM检验结果：")
print(dm_result.to_string(index=False))


# ---------------------- 可视化 ----------------------
print("\n=== 生成MCS vs DQN对比可视化 ===")
plt.figure(figsize=(14, 7))  
# 原有曲线（不变）
plt.plot(test_dates, y_test_inverse, '-', color='black', linewidth=2, label='Actual Price', alpha=0.8)
plt.plot(test_dates, single_pred_inverse, '--', color='tab:blue', linewidth=1.5, label='Single-layer Transformer', alpha=0.7)
plt.plot(test_dates, multi_pred_inverse, '--', color='tab:green', linewidth=1.5, label='Multi-layer Transformer', alpha=0.7)
plt.plot(test_dates, bi_pred_inverse, '--', color='tab:red', linewidth=1.5, label='Bidirectional Transformer', alpha=0.7)
plt.plot(test_dates, svr_pred_inverse, '-.', color='tab:orange', linewidth=1.5, label='SVR', alpha=0.7)
plt.plot(test_dates, dqn_pred_inverse, '-', color='tab:purple', linewidth=2, label='Our DQN-HTS-EF', alpha=0.9)
# 新增MCS曲线
plt.plot(test_dates, mcs_pred_inverse, '-', color='tab:cyan', linewidth=2, label='MCS-Ensemble', alpha=0.9)

# 原有日期处理（不变）
unique_months = []
unique_indices = []
month_set = set()
for index, date in enumerate(pd.to_datetime(test_dates)):
    month_key = date.strftime('%Y-%m')
    if month_key not in month_set:
        unique_months.append(date.strftime('%Y-%m'))
        unique_indices.append(index)
        month_set.add(month_key)

plt.xticks(unique_indices, unique_months, rotation=45)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price', fontsize=12)
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.title(f'Price Prediction: DQN vs MCS ({KEYWORD})', fontsize=14)  
plt.tight_layout()
plt.show()


# ---------------------- MCS和DM结果保存 ----------------------
print("\n=== 保存MCS和DM结果 ===")
# MCS结果保存
mcs_result_data = {
    'MCS信度集': [', '.join(mcs_set)],
    '剔除顺序': [', '.join(elimination_order)],
    '剔除时p值': [', '.join([f'{p:.4f}' for p in p_value_history])],
    'MCS权重': [', '.join([f'{name}:{w:.4f}' for name, w in zip(mcs_set, mcs_weights)])]
}
mcs_result_df = pd.DataFrame(mcs_result_data)
mcs_result_df.to_csv(f"data/{KEYWORD}_mcs_results.csv", index=False)

# DM结果保存
dm_result.to_csv(f"data/{KEYWORD}_dm_test_results.csv", index=False)

# 汇总指标保存（原有+MCS）
summary_metrics = pd.DataFrame([
    {'Model': 'DQN-HTS-EF', 'RMSE': rmse_dqn, 'MSE': mse_dqn, 'R²': r2_dqn, 'MAE': mae_dqn, 'MAPE(%)': mape_dqn},
    {'Model': 'MCS-Ensemble', 'RMSE': rmse_mcs, 'MSE': mse_mcs, 'R²': r2_mcs, 'MAE': mae_mcs, 'MAPE(%)': mape_mcs},
    {'Model': 'Single-Transformer', 'RMSE': rmse_single, 'MSE': mse_single, 'R²': r2_single, 'MAE': mae_single, 'MAPE(%)': mape_single},
    {'Model': 'Multi-Transformer', 'RMSE': rmse_multi, 'MSE': mse_multi, 'R²': r2_multi, 'MAE': mae_multi, 'MAPE(%)': mape_multi},
    {'Model': 'Bi-Transformer', 'RMSE': rmse_bi, 'MSE': mse_bi, 'R²': r2_bi, 'MAE': mae_bi, 'MAPE(%)': mape_bi},
    {'Model': 'SVR', 'RMSE': rmse_svr, 'MSE': mse_svr, 'R²': r2_svr, 'MAE': mae_svr, 'MAPE(%)': mape_svr}
])
summary_metrics.to_csv(f"data/{KEYWORD}_summary_metrics.csv", index=False)

print(f"MCS结果保存至：data/{KEYWORD}_mcs_results.csv")
print(f"DM检验结果保存至：data/{KEYWORD}_dm_test_results.csv")
print(f"汇总指标保存至：data/{KEYWORD}_summary_metrics.csv")